# Preprocessing Neurophysiology Data: Create json-files to Run
[ReTune B04 Dyskinesia Project]


This notebooks helps to create the JSON-files step-by-step, which define 1) which preprocessing settings are used, and 2) which recordings are included in the preprocessing workflow for ECoG- and LFP-data within the ReTune-Project work package B04.


This step-wise structure is provided to understand, visualize, and adjust the single steps. Besides this notebook, a separate py-script provides execution of the preprocessing steps at once via the command line.


<b> Data is required to converted into the BIDS-standard. </b>


### 0. Loading packages and functions, defining paths



In [10]:
# Importing Python and external packages
from os import getcwd, makedirs, chdir
from os.path import join, dirname, exists
import json
import pandas as pd
import numpy as np

In [2]:
def get_project_path_in_notebook(
    subfolder: str = '',
):
    """
    Finds path of projectfolder from Notebook.
    Start running this once to correctly find
    other modules/functions
    """
    path = getcwd()

    while path[-20:] != 'dyskinesia_neurophys':

        path = dirname(path)
    
    return path

In [3]:
# define local storage directories
projectpath = get_project_path_in_notebook()
codepath = join(projectpath, 'code')

# change working directory to project-code folder
chdir(codepath)
getcwd()

'c:\\Users\\habetsj\\Research\\projects\\dyskinesia_neurophys\\code'

In [4]:
from utils.utils_fileManagement import get_project_path, get_onedrive_path
import lfpecog_preproc.preproc_data_management as dataMng

Fix TSV

Use in case a scans_tsv file has to be edited

In [25]:
sub = '019'
bids_sub='EL019'
onedrive_data_path = get_onedrive_path('data')
sub_json_path = join(onedrive_data_path, 'preprocess_jsons',
                    f'runInfo_{sub}.json')
sub_runs = {}
     
with open(sub_json_path) as f:
     sub_json = json.load(f, )  # list of runinfo-dicts

scans_path = join(get_onedrive_path('bids_rawdata'),
                             f'sub-{bids_sub}',
                             f'ses-{sub_json["ses"]}',
                             f'sub-{bids_sub}_ses-{sub_json["ses"]}_scans.tsv')

# scans_pathTEST = join(get_onedrive_path('bids_rawdata'),
#                              f'sub-{bids_sub}',
#                              f'ses-{sub_json["ses"]}',
#                              f'sub-{bids_sub}_ses-{sub_json["ses"]}_scansTEST.tsv')

scans_df = pd.read_csv(scans_path, sep='\t',)

new_df = pd.DataFrame(columns=['filename', 'acq_time'],
                      data=[['', '']] * scans_df.shape[0])

for i in np.arange(scans_df.shape[0]):

     s = scans_df['filename  acq_time'][i]
     s1, s2 = s.split(' ')
     new_df['filename'][i] = s1
     new_df['acq_time'][i] = s2

new_df.to_csv(scans_path, sep='\t', header=True, index=False)

Create JSON

In [31]:
json_fname = 'preprocSettings_v3.1.json'

json_folder = join(get_onedrive_path('data'), 'preprocess_jsons')
json_path = join(json_folder, json_fname)

with open(json_path, 'r') as json_data:

    mainSettings = json.load(json_data)  # gets dir

for sub in mainSettings['subs_include']:

    if sub == '014': continue

    sub_runs = dataMng.get_sub_runs(sub)

    for run in list(sub_runs.values()):
    
        if 'dopa' not in run['acq'].lower():
            # print(f'\n\tRun {run} SKIPPED, NO "DOPA" IN NAME')
            continue

        print(f'\nSTART PREPROCESSING Run: {run}\n')

        runInfo = dataMng.RunInfo(
            mainSettings=mainSettings,
            runDict=run,
            project_path=projectpath,
        )
        
        rawRun = dataMng.defineMneRunData(
            runInfo=runInfo,
            subSettings=run,
        )

In [36]:
rawRun.acc.ch_names

['ACC_R_X_D2_TM',
 'ACC_R_Y_D2_TM',
 'ACC_R_Z_D2_TM',
 'ACC_L_X_D2_TM',
 'ACC_L_Y_D2_TM',
 'ACC_L_Z_D2_TM']

In [37]:
runInfo.bidspath

BIDSPath(
root: c:/Users/habetsj/OneDrive - Charité - Universitätsmedizin Berlin/BIDS_Berlin_ECOG_LFP/rawdata
datatype: ieeg
basename: sub-EL012_ses-EcogLfpMedOffOnDys01_task-Free_acq-StimOffDopa50_run-1_ieeg.vhdr)

In [3]:
# Import own functions
import lfpecog_preproc.preproc_json_creator as json_creator


### 1. Define Settings of Preprocessing and JSON-file

In [4]:
def define_settings(
    version: str, create: bool, json_dir: str,
    ecog_sett_list: list = None, lfp_sett_list: list = None,
    plot_figures: bool = False, lfp_reref: str = None
):
    '''
    Function...

    Arguments:
        - version: name of settings-version
        - create: To create new or not (used existing)
        - json_dir: path to json-folder
        - ecog_sett_list/ lfp_sett_list: Lists with new settings
        to make for ecog and lfp preprocessing:
        [win_len, artfct_sd_tresh,
        (bandpass_f_low, bandpass_f_low), transBW, notchW,
        Fs_origin, Fs_resample, settings_version]

    '''
    f = f'settings_{version}.json'
    if create:
        # check for already existing
        if f in os.listdir(json_dir):
                print(f'\nSettings-JSON {version} does already exist,'
                ' define new name for current version')
                return
        for newlist in [ecog_sett_list, lfp_sett_list]:
            if newlist is None:  # check if new list is inserted
                print('Enter list(s) with new settings')
                return
            if (float or str) in [type(s) for s in newlist]:  # only int
                print('List with new settings can only contain integers')
                return
        if lfp_reref not in ['segments', 'levels']:
            print('LFP-reref method has to be levels / segments')
            return

        # if all requirements are met: create new json
        dict_settings = {  # dict to write into json
            'lfp': lfp_sett_list + [version],
            'ecog': ecog_sett_list + [version],
            'plot_figs': plot_figures,
            'lfp_reref': lfp_reref,
        }
        with open(os.path.join(json_dir, f), 'w') as jsonfile:
            json.dump(dict_settings, jsonfile, indent=4)
        return os.path.join(json_dir, f)

    if not create:
        # Define existing version to use
        if f not in os.listdir(json_dir):
            print(f'Setting-json {version} does not exist.\n'
                  'Insert existing settings-version!!')
            return
        else:
            print(f'Settings {version} exist, please proceed :)')
            return os.path.join(json_dir, f)


In [5]:
lfp_settings = [1024, 4, (1, 120), 10, 2, 4000, 800,]
ecog_settings = [1024, 4, (1, 120), 10, 2, 4000, 800,]

json_settings = define_settings(
    'v0.6_Feb22', create=True, json_dir=json_dir,
    ecog_sett_list=ecog_settings, lfp_sett_list=lfp_settings,
    plot_figures=True, lfp_reref='levels'
)

### 2. Select Recordings to Preprocess




#### a) Print available recordings in session

In [25]:
# define sub and ses and check available runs
sub = '008'
ses = 'MedOn02'
ses_path = os.path.join(
    rawdatapath,
    f'sub-{sub}',
    f'ses-Ephys{ses}/ieeg'  #EcogLfp
)
files = [f for f in os.listdir(ses_path) if f[-4:] == '.eeg']
print(f'Available runs for pt {sub} in session {ses}:')
for n, f in enumerate(files): print('\n',n, f.split('_')[2:5])


Available runs for pt 008 in session MedOn02:

 0 ['task-Rest', 'acq-StimOffDopa00', 'run-1']

 1 ['task-SelfpacedHandTapL', 'acq-StimOffDopa60', 'run-1']

 2 ['task-SelfpacedHandTapL', 'acq-StimOffDopa15', 'run-1']

 3 ['task-Rest', 'acq-StimOffDopa10', 'run-1']

 4 ['task-Free', 'acq-StimOffDopa55', 'run-1']

 5 ['task-SelfpacedHandTapL', 'acq-StimOffDopa35', 'run-1']

 6 ['task-Rest', 'acq-StimOffDopa30', 'run-1']

 7 ['task-Free', 'acq-StimOffDopa20', 'run-1']

 8 ['task-Rest', 'acq-StimOffDopa50', 'run-1']


#### b) Choose which recordings to exclude from Preprocessing execution

In [26]:
# Select files based on numbers above
print('Excluded from Preprocessing:')
num_sel_excl = [7]  # add numbers here, or leave empty to include all recordings
to_excl = []  # leave empty
for f_ex in num_sel_excl:
    to_excl.append(files[f_ex])
    print(f'{files[f_ex]} added to exclude from preprocessing')

# Can not be run twice!! First re-run the cell above to reset the numbers again

Exclusion from Preprocessing:
sub-008_ses-EphysMedOn02_task-Free_acq-StimOffDopa20_run-1_ieeg.eeg added to exclude from preprocessing


In [27]:
# Check included files after removal
for f_ex in to_excl: files.remove(f_ex)
print('Included for Preprocessing:')
for f in files:
    print(f'\n{f}')


Included for Preprocessing:

sub-008_ses-EphysMedOn02_task-Rest_acq-StimOffDopa00_run-1_ieeg.eeg

sub-008_ses-EphysMedOn02_task-SelfpacedHandTapL_acq-StimOffDopa60_run-1_ieeg.eeg

sub-008_ses-EphysMedOn02_task-SelfpacedHandTapL_acq-StimOffDopa15_run-1_ieeg.eeg

sub-008_ses-EphysMedOn02_task-Rest_acq-StimOffDopa10_run-1_ieeg.eeg

sub-008_ses-EphysMedOn02_task-Free_acq-StimOffDopa55_run-1_ieeg.eeg

sub-008_ses-EphysMedOn02_task-SelfpacedHandTapL_acq-StimOffDopa35_run-1_ieeg.eeg

sub-008_ses-EphysMedOn02_task-Rest_acq-StimOffDopa30_run-1_ieeg.eeg

sub-008_ses-EphysMedOn02_task-Rest_acq-StimOffDopa50_run-1_ieeg.eeg


#### c) Create dict's and write them into JSON-file

In [28]:
# convert files into dictionaries to write into JSON
list_dicts = []
for f in files:
    list_dicts.append({  # dict to write into json
        'sub': f.split('-')[1][:3],
        'ses': f.split('-')[2][:-5],
        'task': f.split('-')[3][:-4],
        'acq': f.split('-')[4][:-4],
        'run': f.split('-')[5][0],
        'raw_path': rawdatapath,
        'project_path': projectpath,
    })

print('Check dictionaries of included runs:')
for d in list_dicts: print(f'\n{d}')

Check dictionaries of included runs:

{'sub': '008', 'ses': 'EphysMedOn02', 'task': 'Rest', 'acq': 'StimOffDopa00', 'run': '1', 'raw_path': '/Users/jeroenhabets/OneDrive - Charité - Universitätsmedizin Berlin/BIDS_Berlin_ECOG_LFP/rawdata', 'project_path': '/Users/jeroenhabets/Research/CHARITE/projects/dyskinesia_neurophys'}

{'sub': '008', 'ses': 'EphysMedOn02', 'task': 'SelfpacedHandTapL', 'acq': 'StimOffDopa60', 'run': '1', 'raw_path': '/Users/jeroenhabets/OneDrive - Charité - Universitätsmedizin Berlin/BIDS_Berlin_ECOG_LFP/rawdata', 'project_path': '/Users/jeroenhabets/Research/CHARITE/projects/dyskinesia_neurophys'}

{'sub': '008', 'ses': 'EphysMedOn02', 'task': 'SelfpacedHandTapL', 'acq': 'StimOffDopa15', 'run': '1', 'raw_path': '/Users/jeroenhabets/OneDrive - Charité - Universitätsmedizin Berlin/BIDS_Berlin_ECOG_LFP/rawdata', 'project_path': '/Users/jeroenhabets/Research/CHARITE/projects/dyskinesia_neurophys'}

{'sub': '008', 'ses': 'EphysMedOn02', 'task': 'Rest', 'acq': 'S

In [29]:
# Write JSON-file with run-dict's
fname = 'runinfos_11FEB22b.json'
f = os.path.join(json_dir, fname)
with open(f, 'w') as jsonfile:
    json.dump(list_dicts, jsonfile, indent=4)